# **Projet IA: Suivi de la réussite étudiante**

 

**<u>Objectif:</u>**  Déterminer les étudiants à risque d'échec afin de mieux les accompagner en vue de leur réussite académique; 

* Identifier les facteurs importants qui affectent les performances académiques de l'étudiant 
* Prédiction de la note à l'examen finale de l'étudiant 
* Classifier les étudiants en catégories potentielles "Réussite" ou "Échec" 

**<u>Description des caractéristiques:</u>**
* **Student_ID :** Identifiant unique pour chaque étudiant. 

* **Gender :** Sexe de l'étudiant (Homme/Femme). 
* **Study_Hours_per_Week :** Nombre moyen d'heures d'étude par semaine. 
* **Attendance_Rate :** Pourcentage d'assiduité (50% - 100%). 
* **Past_Exam_Scores :** Note moyenne des examens précédents (50 - 100). 
* **Parental_Education_Level :** Niveau d'éducation des parents (Lycée, Licence, Master, Doctorat). 
* **Internet_Access_at_Home :** Indique si l'étudiant a un accès à Internet à la maison (Oui/Non). 
* **Extracurricular_Activities :** Indique si l'étudiant participe à des activités parascolaires (Oui/Non). 
* **Final_Exam_Score (Target) :** La note à l'examen final de l'étudiant (50 - 100, valeurs entières). 
* **Pass_Fail (Target) :** Le statut de l'étudiant (Réussite/Échec). 

## **Analyse Exploratoire des Données (EDA)**

1. **Compréhension des données**
   * Aperću général (dimension du dataset, types de variables, résumé statistiques et autres)
   * Analyse univariée (Étudier les variables une par une)
   * Analyse bivariée (Explorer les relations entre les variables)
   * Détection d'anomalies (Identifier les outliers, valeurs extrêmes ou abérantes)
   * Valeurs manquantes (Réperer les colonnes ou lignes incomplètes)
   * Distribution du jeu de données (Comment les données sont réparties)

In [5]:
# Charger le jeu de données
import pandas as pd
students_df = pd.read_csv("../data/raw/student_performance_dataset.csv")
students_df.head()

,Student_ID,Gender,Study_Hours_per_Week,Attendance_Rate,Past_Exam_Scores,Parental_Education_Level,Internet_Access_at_Home,Extracurricular_Activities,Final_Exam_Score,Pass_Fail
0,S147,Male,31,68.267841,86,High School,Yes,Yes,63,Pass
1,S136,Male,16,78.222927,73,PhD,No,No,50,Fail
2,S209,Female,21,87.525096,74,PhD,Yes,No,55,Fail
3,S458,Female,27,92.076483,99,Bachelors,No,No,65,Pass
4,S078,Female,37,98.655517,63,Masters,No,Yes,70,Pass


In [6]:
# La colonne 'Student_ID' est superflue, non importante
students_df = students_df.drop(columns=['Student_ID'])
students_df.head()

,Gender,Study_Hours_per_Week,Attendance_Rate,Past_Exam_Scores,Parental_Education_Level,Internet_Access_at_Home,Extracurricular_Activities,Final_Exam_Score,Pass_Fail
0,Male,31,68.267841,86,High School,Yes,Yes,63,Pass
1,Male,16,78.222927,73,PhD,No,No,50,Fail
2,Female,21,87.525096,74,PhD,Yes,No,55,Fail
3,Female,27,92.076483,99,Bachelors,No,No,65,Pass
4,Female,37,98.655517,63,Masters,No,Yes,70,Pass


In [11]:
# Informations du jeu de données
students_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gender                      708 non-null    object 
 1   Study_Hours_per_Week        708 non-null    int64  
 2   Attendance_Rate             708 non-null    float64
 3   Past_Exam_Scores            708 non-null    int64  
 4   Parental_Education_Level    708 non-null    object 
 5   Internet_Access_at_Home     708 non-null    object 
 6   Extracurricular_Activities  708 non-null    object 
 7   Final_Exam_Score            708 non-null    int64  
 8   Pass_Fail                   708 non-null    object 
dtypes: float64(1), int64(3), object(5)
memory usage: 49.9+ KB


In [12]:
students_df.shape # La taille de notre df

(708, 9)

In [13]:
# Résumé statistique des données numériques
students_df.describe()

,Study_Hours_per_Week,Attendance_Rate,Past_Exam_Scores,Final_Exam_Score
count,708.000000,708.000000,708.000000,708.000000
mean,26.132768,78.107722,77.871469,58.771186
std,8.877727,13.802802,14.402739,6.705877
min,10.000000,50.116970,50.000000,50.000000
25%,19.000000,67.550094,65.000000,52.000000
50%,27.000000,79.363046,79.000000,59.500000
75%,34.000000,89.504232,91.000000,64.000000
max,39.000000,99.967675,100.000000,77.000000


In [28]:
# Identifions les colonnes catégorielles des numériques
numerical_col = students_df.select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_col = [col for col in students_df.columns if students_df[col].dtype == 'object']
students_df_col = numerical_col + categorical_col # La fusion des 02.
students_df_col

['Study_Hours_per_Week',
 'Attendance_Rate',
 'Past_Exam_Scores',
 'Final_Exam_Score',
 'Gender',
 'Parental_Education_Level',
 'Internet_Access_at_Home',
 'Extracurricular_Activities',
 'Pass_Fail']

In [29]:
# Déterminer les valeurs manquantes
students_df.isna().sum()

Gender                        0
Study_Hours_per_Week          0
Attendance_Rate               0
Past_Exam_Scores              0
Parental_Education_Level      0
Internet_Access_at_Home       0
Extracurricular_Activities    0
Final_Exam_Score              0
Pass_Fail                     0
dtype: int64

In [35]:
# Déterminer s'il existe des données dupliquées
students_df.duplicated().sum()

np.int64(208)

In [37]:
# Filtrer les données dupliquées
students_df[students_df.duplicated()] # Retourne toutes les données qui dupliquées

,Gender,Study_Hours_per_Week,Attendance_Rate,Past_Exam_Scores,Parental_Education_Level,Internet_Access_at_Home,Extracurricular_Activities,Final_Exam_Score,Pass_Fail
20,Female,27,92.076483,99,Bachelors,No,No,65,Pass
42,Female,29,68.864298,92,High School,No,No,64,Pass
49,Female,20,89.075724,92,Bachelors,Yes,Yes,62,Pass
64,Male,39,81.662039,97,Bachelors,Yes,Yes,66,Pass
90,Female,15,77.836079,95,Masters,No,Yes,61,Pass
...,...,...,...,...,...,...,...,...,...
692,Female,21,96.148012,84,Bachelors,Yes,No,65,Pass
695,Female,33,87.298726,73,Masters,No,Yes,72,Pass
697,Female,35,68.461524,99,PhD,Yes,No,67,Pass
704,Male,35,60.278990,83,Masters,No,No,62,Pass


Tout à fait compréhensible, ces données se retrouvent dupliquées car plusieurs étudiants pourraient avoir les mêmes comportements au niveau scolaire, nombre d'heures de révisions par semaines, mêmes notes etc.

In [42]:
# Déterminons la proportion d'une cible
students_df['Pass_Fail'].value_counts().reset_index()

,Pass_Fail,count
0,Pass,354
1,Fail,354


In [43]:
# Donnée très équilibrée,s'assurer de maintenir cette proportion lors de la séparation du jeu de données